### Start

In [1]:
import numpy as np
from osgeo import gdal
from osgeo import osr
import time
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from tensorflow.keras import backend as K

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(devices=physical_devices[0], device_type='GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
keras.backend.set_image_data_format('channels_last')

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
#自定义matries 计算r2
def R2(y_true, y_pred):
    sst = K.sum(K.square(y_true - K.mean(y_true)))
    ssr = K.sum(K.square(y_pred - y_true))
    R2 = 1 - ssr/sst
    return R2

## Helper functions

In [6]:
def Array2Image(data,lats,lons):
  # get the unique coordinates
  uniqueLats = np.unique(lats)
  uniqueLons = np.unique(lons)
 
  # get number of columns and rows from coordinates
  ncols = len(uniqueLons)
  nrows = len(uniqueLats)
 
  # determine pixelsizes
  ys = uniqueLats[1] - uniqueLats[0]  
  xs = uniqueLons[1] - uniqueLons[0]
 
  # create an array with dimensions of image
  # 这里把缺失值设置成了0，之后需要修改
  arr = np.zeros([nrows, ncols], np.float32) #-9999
  arr.fill(np.nan)
 
  # fill the array with values
  for counter in range(0,len(data),1):
    index_lon = np.where(uniqueLons == lons[counter])[0][0]
    index_lat = np.where(uniqueLats == lats[counter])[0][0]
    arr[len(uniqueLats)-1-index_lat,index_lon] = data[counter]
  return arr

In [7]:
def ReadData(df:pd.DataFrame):
    allData = []
    for j in range(df.shape[1]):
        data = []
        for i in range(df.shape[0]):
            #print(df.iloc[i,j])
            data_i = gdal.Open(df.iloc[i,j])
            band_i = data_i.GetRasterBand(1)
            array_i = band_i.ReadAsArray()
            #array_i[array_i == noData] = np.nan
            data.append(np.reshape(array_i,(-1)))
        allData.append(data)
    allData = np.array(allData,dtype = np.float64).squeeze()
    return pd.DataFrame(allData.T,columns=df.columns.to_list())

In [8]:
templete  = gdal.Open('F:/SGYL/SM_Downscaling2/Lon.tif')
def convert2Tif(data,outputPath):
    driver = gdal.GetDriverByName('GTiff')
    tif = driver.Create(outputPath,data.shape[1],data.shape[0],1,gdal.GDT_Int16)
    #geotransform = (73.49117339381806,0.008983152841195215,0.0,39.831299697859585,0.0,-0.008983152841195215)
    tif.SetGeoTransform(templete.GetGeoTransform())
    tif.SetProjection(templete.GetProjection())
    tif.GetRasterBand(1).WriteArray(data)
    #tif.GetRasterBand(1).SetNoDataValue()
    tif.FlushCache()
    del tif


## Preprocessing

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
noData_i = gdal.Open('F:/SGYL/SM_results_data/DATA/data_2020/LST_Diff/LST_Diff_2020_1.tif')
noData_band = noData_i.GetRasterBand(1)
noData_arr = noData_band.ReadAsArray()
noData = np.min(noData_arr)
print(noData)

-3.4e+38


## Predict

In [11]:
#['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'dem', 'aspect', 'slope', 'clay', 'sand', 'silt', 'Lon', 'Lat']
error_index_year = []
error_index_day = []
batch_size = 4096*4
for year in range(2001,2002):
    try:
        print(year,'************************************************')
        print('load model')
        model_path = os.path.join('D:/SGYL/SM_results_data/check_points/ANN/','ANN_'+str(year)+'_best.hdf5')
        print(model_path)
        model = keras.models.load_model(model_path,custom_objects={'R2':R2})

        print('load training data')
        #load training data
        data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
        X_train = data_train.drop(['SM','Aspect'],axis = 1)
        X_train_columns = X_train.columns.to_list()
        print(X_train.columns.to_list())
        standarder = StandardScaler()
        X_train = standarder.fit_transform(X_train)

        print('prepare file path')
        # prepare data path
        feature_order = ['NDVI','EVI','LST','LST_Diff','Pre','SWCI','VSDI','SIWSI','ET'] #
        static_feature = ['TWI','dem','slope','clay','sand','silt','Lon','Lat']
        print(feature_order+static_feature)
        root = 'F:/SGYL/SM_results_data/DATA/data_'+str(year)+'/'

        predictor_path = dict()
        for dir in feature_order:
            path = os.path.join(root,dir)
            if dir == 'ET':
                files = [os.path.join(path+'/',i) for i in os.listdir(path+'/')]
            elif year in [2004,2008,2012,2016,2020]:
                files = [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,367)]
            elif year == 2001:
                files = [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,362)]
            else:
                files = [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]
            predictor_path[dir] = files

        path_length = len(predictor_path['LST'])

        for var in static_feature:
            path_i = os.path.join('F:/SGYL/SM_results_data/Static/',var+'.tif')
            files = [path_i for i in range(path_length)]
            predictor_path[var] = files

        predictor_path = pd.DataFrame(predictor_path)
        predictor_path.columns = feature_order+static_feature
        
        print('start predict','************************************************')       
        
        for i in range(323,predictor_path.shape[0]):#predictor_path.shape[0]
            try:
                #print(i,'************************************************')
                data_i = ReadData(predictor_path.iloc[i:(i+1),:])
                lon_lat = data_i[['Lon','Lat']]
                data_i['DOY'] = int(i+1)
                data_i.columns = X_train_columns
                data_i = data_i.to_numpy()
                data_i[data_i == noData] = np.nan
                    #print(data_i.shape)
                    #dealing with missing values median
                for index in range(data_i.shape[1]):
                    median_i = np.nanmedian(data_i[:,index])
                    data_i[:,index][np.isnan(data_i[:,index])] = median_i
                data_i = pd.DataFrame(data_i,columns=X_train_columns)
                data_i = standarder.transform(data_i)
                    # XGBoost 不需要reshape
                data_i = data_i.reshape(data_i.shape[0],data_i.shape[1],1)
                data_ii = [data_i[i*batch_size:(i+1)*batch_size,:,:] for i in range(data_i.shape[0]//batch_size+1)]
                data_pred = np.concatenate([model(i,training = False).numpy() for i in data_ii])#model.predict(data_i,batch_size=4096)
                #print(data_pred.shape)
                predicted_arr = Array2Image(data_pred,lon_lat['Lat'],lon_lat['Lon'])
                save_path = 'F:/SGYL/SM_results_data/results/'+str(year)+'/'+'ANN'+'/'+'ANN_'+str(year)+'_'+str(i+1)+'.tif'
                predicted_arr = predicted_arr * 10000
                convert2Tif(predicted_arr,save_path)
                print(i,': save the %d tif successfully for the year %d '%(i+1,year))
            except Exception as e:
                print('failed at year %d index %d'%(year,i+1)) # 这里的index是第几天
                print(repr(e))
                error_index_day.append((year,i))
            continue
    except:
        print('failed at index %d'%(i+1)) # 这里的index是第几天
        error_index_year.append((year,i))
    continue
    

2001 ************************************************
load model
D:/SGYL/SM_results_data/check_points/ANN/ANN_2001_best.hdf5
load training data
['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'DEM', 'Slope', 'Clay', 'Sand', 'Silt', 'Lon', 'Lat', 'DOY']
prepare file path
['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'dem', 'slope', 'clay', 'sand', 'silt', 'Lon', 'Lat']
start predict ************************************************
323 : save the 324 tif successfully for the year 2001 
324 : save the 325 tif successfully for the year 2001 
325 : save the 326 tif successfully for the year 2001 
326 : save the 327 tif successfully for the year 2001 
327 : save the 328 tif successfully for the year 2001 
328 : save the 329 tif successfully for the year 2001 
329 : save the 330 tif successfully for the year 2001 
330 : save the 331 tif successfully for the year 2001 
331 : save the 332 tif successfully for the year 2001 
332 :

### here

In [12]:
#print(error_index_year)
print(error_index_day)

[]


In [13]:
#['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'dem', 'aspect', 'slope', 'clay', 'sand', 'silt', 'Lon', 'Lat']
error_index_year = []
error_index_day = []
for year in [2012]:
    try:
        print(year,'************************************************')
        print('load model')
        model_path = os.path.join('D:/SGYL/SM_results_data/check_points/ANN/','ANN_'+str(year)+'_best.hdf5')
        print(model_path)
        model = keras.models.load_model(model_path,custom_objects={'R2':R2})

        print('load training data')
        #load training data
        data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
        X_train = data_train.drop(['SM','Aspect'],axis = 1)
        X_train_columns = X_train.columns.to_list()
        print(X_train.columns.to_list())
        standarder = StandardScaler()
        X_train = standarder.fit_transform(X_train)

        print('prepare file path')
        # prepare data path
        feature_order = ['NDVI','EVI','LST','LST_Diff','Pre','SWCI','VSDI','SIWSI','ET'] #
        static_feature = ['TWI','dem','slope','clay','sand','silt','Lon','Lat']
        print(feature_order+static_feature)
        root = 'F:/SGYL/SM_results_data/DATA/data_'+str(year)+'/'

        predictor_path = dict()
        for dir in feature_order:
            path = os.path.join(root,dir)
            if dir == 'ET':
                files = [os.path.join(path+'/',i) for i in os.listdir(path+'/')]
            elif year in [2004,2008,2012,2016,2020]:
                files = [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,367)]
            else:
                files = [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]
            predictor_path[dir] = files

        path_length = len(predictor_path['LST'])

        for var in static_feature:
            path_i = os.path.join('F:/SGYL/SM_results_data/Static/',var+'.tif')
            files = [path_i for i in range(path_length)]
            predictor_path[var] = files

        predictor_path = pd.DataFrame(predictor_path)
        predictor_path.columns = feature_order+static_feature
        
        print('start predict','************************************************')       
        
        for i in range(predictor_path.shape[0]):#predictor_path.shape[0]
            try:
                #print(i,'************************************************')
                data_i = ReadData(predictor_path.iloc[i:(i+1),:])
                lon_lat = data_i[['Lon','Lat']]
                data_i['DOY'] = int(i+1)
                data_i.columns = X_train_columns
                data_i = data_i.to_numpy()
                data_i[data_i == noData] = np.nan
                    #print(data_i.shape)
                    #dealing with missing values median
                for index in range(data_i.shape[1]):
                    median_i = np.nanmedian(data_i[:,index])
                    data_i[:,index][np.isnan(data_i[:,index])] = median_i
                data_i = pd.DataFrame(data_i,columns=X_train_columns)
                data_i = standarder.transform(data_i)
                    # XGBoost 不需要reshape
                data_i = data_i.reshape(data_i.shape[0],data_i.shape[1],1)
                data_pred = model.predict(data_i,batch_size=4096*4)
                #print(data_pred.shape)
                predicted_arr = Array2Image(data_pred,lon_lat['Lat'],lon_lat['Lon'])
                save_path = 'F:/SGYL/SM_results_data/results/'+str(year)+'/'+'ANN'+'/'+'ANN_'+str(year)+'_'+str(i+1)+'.tif'
                predicted_arr = predicted_arr * 10000
                convert2Tif(predicted_arr,save_path)
                print(i,': save the %d tif successfully for the year %d '%(i+1,year))
            except Exception as e:
                print('failed at year %d index %d'%(year,i+1)) # 这里的index是第几天
                print(repr(e))
                error_index_day.append((year,i))
            continue
    except:
        print('failed at index %d'%(i+1)) # 这里的index是第几天
        error_index_year.append((year,i))
    continue
    

2012 ************************************************
load model
D:/SGYL/SM_results_data/check_points/ANN/ANN_2012_best.hdf5
load training data
['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'DEM', 'Slope', 'Clay', 'Sand', 'Silt', 'Lon', 'Lat', 'DOY']
prepare file path
['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'dem', 'slope', 'clay', 'sand', 'silt', 'Lon', 'Lat']
start predict ************************************************
0 : save the 1 tif successfully for the year 2012 
1 : save the 2 tif successfully for the year 2012 
2 : save the 3 tif successfully for the year 2012 
3 : save the 4 tif successfully for the year 2012 
4 : save the 5 tif successfully for the year 2012 
5 : save the 6 tif successfully for the year 2012 
6 : save the 7 tif successfully for the year 2012 
7 : save the 8 tif successfully for the year 2012 
8 : save the 9 tif successfully for the year 2012 
9 : save the 10 tif successfully for the 

In [14]:
#print(error_index_year)
print(error_index_day)

[]


In [17]:
#['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'dem', 'aspect', 'slope', 'clay', 'sand', 'silt', 'Lon', 'Lat']
error_index_year = []
error_index_day = []
for year in [2013]:
    try:
        print(year,'************************************************')
        print('load model')
        model_path = os.path.join('D:/SGYL/SM_results_data/check_points/ANN/','ANN_'+str(year)+'_best.hdf5')
        print(model_path)
        model = keras.models.load_model(model_path,custom_objects={'R2':R2})

        print('load training data')
        #load training data
        data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
        X_train = data_train.drop(['SM','Aspect'],axis = 1)
        X_train_columns = X_train.columns.to_list()
        print(X_train.columns.to_list())
        standarder = StandardScaler()
        X_train = standarder.fit_transform(X_train)

        print('prepare file path')
        # prepare data path
        feature_order = ['NDVI','EVI','LST','LST_Diff','Pre','SWCI','VSDI','SIWSI','ET'] #
        static_feature = ['TWI','dem','slope','clay','sand','silt','Lon','Lat']
        print(feature_order+static_feature)
        root = 'F:/SGYL/SM_results_data/DATA/data_'+str(year)+'/'

        predictor_path = dict()
        for dir in feature_order:
            path = os.path.join(root,dir)
            if dir == 'ET':
                files = [os.path.join(path+'/',i) for i in os.listdir(path+'/')]
            elif year in [2004,2008,2012,2016,2020]:
                files = [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,367)]
            else:
                files = [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]
            predictor_path[dir] = files

        path_length = len(predictor_path['LST'])

        for var in static_feature:
            path_i = os.path.join('F:/SGYL/SM_results_data/Static/',var+'.tif')
            files = [path_i for i in range(path_length)]
            predictor_path[var] = files

        predictor_path = pd.DataFrame(predictor_path)
        predictor_path.columns = feature_order+static_feature
        
        print('start predict','************************************************')       
        
        for i in range(predictor_path.shape[0]):#predictor_path.shape[0]
            try:
                #print(i,'************************************************')
                data_i = ReadData(predictor_path.iloc[i:(i+1),:])
                lon_lat = data_i[['Lon','Lat']]
                data_i['DOY'] = int(i+1)
                data_i.columns = X_train_columns
                data_i = data_i.to_numpy()
                data_i[data_i == noData] = np.nan
                    #print(data_i.shape)
                    #dealing with missing values median
                for index in range(data_i.shape[1]):
                    median_i = np.nanmedian(data_i[:,index])
                    data_i[:,index][np.isnan(data_i[:,index])] = median_i
                data_i = pd.DataFrame(data_i,columns=X_train_columns)
                data_i = standarder.transform(data_i)
                    # XGBoost 不需要reshape
                data_i = data_i.reshape(data_i.shape[0],data_i.shape[1],1)
                data_pred = model.predict(data_i,batch_size=4096*4)
                #print(data_pred.shape)
                predicted_arr = Array2Image(data_pred,lon_lat['Lat'],lon_lat['Lon'])
                save_path = 'F:/SGYL/SM_results_data/results/'+str(year)+'/'+'ANN'+'/'+'ANN_'+str(year)+'_'+str(i+1)+'.tif'
                predicted_arr = predicted_arr * 10000
                convert2Tif(predicted_arr,save_path)
                print(i,': save the %d tif successfully for the year %d '%(i+1,year))
            except Exception as e:
                print('failed at year %d index %d'%(year,i+1)) # 这里的index是第几天
                print(repr(e))
                error_index_day.append((year,i))
            continue
    except:
        print('failed at index %d'%(i+1)) # 这里的index是第几天
        error_index_year.append((year,i))
    continue
    

2013 ************************************************
load model
D:/SGYL/SM_results_data/check_points/ANN/ANN_2013_best.hdf5
load training data
['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'DEM', 'Slope', 'Clay', 'Sand', 'Silt', 'Lon', 'Lat', 'DOY']
prepare file path
['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'dem', 'slope', 'clay', 'sand', 'silt', 'Lon', 'Lat']
start predict ************************************************
0 : save the 1 tif successfully for the year 2013 


In [16]:
print(error_index_day)

[(2013, 29), (2013, 30), (2013, 31), (2013, 32), (2013, 33), (2013, 34), (2013, 35), (2013, 36), (2013, 37), (2013, 38), (2013, 39), (2013, 40), (2013, 41), (2013, 42), (2013, 43), (2013, 44), (2013, 45), (2013, 46), (2013, 47), (2013, 48), (2013, 49), (2013, 50), (2013, 51), (2013, 52), (2013, 53), (2013, 54), (2013, 55), (2013, 56), (2013, 57), (2013, 58), (2013, 59), (2013, 60), (2013, 61), (2013, 62), (2013, 63), (2013, 64), (2013, 65), (2013, 66), (2013, 67), (2013, 68), (2013, 69), (2013, 70), (2013, 71), (2013, 72), (2013, 73), (2013, 74), (2013, 75), (2013, 76), (2013, 77), (2013, 78), (2013, 79), (2013, 80), (2013, 81), (2013, 82), (2013, 83), (2013, 84), (2013, 85), (2013, 86), (2013, 87), (2013, 88), (2013, 89), (2013, 90), (2013, 91), (2013, 92), (2013, 93), (2013, 94), (2013, 95), (2013, 96), (2013, 97), (2013, 98), (2013, 99), (2013, 100), (2013, 101), (2013, 102), (2013, 103), (2013, 104), (2013, 105), (2013, 106), (2013, 107), (2013, 108), (2013, 109), (2013, 110), (201

In [ ]:
test = gdal.Open('E:/SM_results_2/25km_ESACCI/allyear/clipped/LST/LST_D/LST_D_final_2001_2001.tif')

In [ ]:
alldata = []
for i in range(test.RasterCount):
    band_i = test.GetRasterBand((i+1))
    array_i = band_i.ReadAsArray()
    alldata.append(array_i)
alldata = np.array(alldata,dtype= np.float64)
alldata.shape

In [ ]:
np.where(np.isnan(alldata))